In [1]:
#Importing required libraries for data cleanup on CSV files, and to create DataFrames

import pandas as pd
from pathlib import Path
import numpy as np
import csv
from datetime import datetime

In [2]:
#Creating Dataframes from the CSV files
inlaw_df = pd.read_csv(Path("Updated_CSV/inlaw_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date')
johnson_df = pd.read_csv(Path("Updated_CSV/johnson_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date') 
johnson_ira_df = pd.read_csv(Path("Updated_CSV/johnson_ira_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date')
inlaw_money_movement_df = pd.read_csv(Path("Updated_CSV/inlaw_money_movement_csv"), infer_datetime_format=True, parse_dates=True, )
johnson_money_movement_df = pd.read_csv(Path("Updated_CSV/johnson_money_movement_csv"), infer_datetime_format=True, parse_dates=True)
johnson_ira_money_movement_df = pd.read_csv(Path("Updated_CSV/johnson_ira_money_movement_csv"), infer_datetime_format=True, parse_dates=True)

In [3]:
#Changing Dates to datetime formats
inlaw_df['Expiration Date'] = pd.to_datetime(inlaw_df['Expiration Date'])
johnson_df['Expiration Date'] = pd.to_datetime(johnson_df['Expiration Date'])
johnson_ira_df['Expiration Date'] = pd.to_datetime(johnson_ira_df['Expiration Date'])
inlaw_money_movement_df['Date'] = pd.to_datetime(inlaw_money_movement_df['Date'])
johnson_money_movement_df['Date'] = pd.to_datetime(johnson_money_movement_df['Date'])
johnson_ira_money_movement_df['Date'] = pd.to_datetime(johnson_ira_money_movement_df['Date'])

In [4]:
#Finding total new deposits for all 3 money movement accounts
inlaw_net_deposits = inlaw_money_movement_df['Value'].sum()
johnson_net_deposits = round(johnson_money_movement_df['Value'].sum(),2)
johnson_ira_net_deposits = johnson_ira_money_movement_df['Value'].sum()

In [5]:
#Calculating start, end, and age of all 3 accounts
end_date_inlaw=inlaw_money_movement_df['Date'][0]
end_date_johnson=johnson_money_movement_df['Date'][0]
end_date_johnson_ira=johnson_ira_money_movement_df['Date'][0]
start_date_inlaw=inlaw_money_movement_df['Date'].iloc[-1]
start_date_johnson=johnson_money_movement_df['Date'].iloc[-1]
start_date_johnson_ira=johnson_ira_money_movement_df['Date'].iloc[-1]
age_of_account_inlaw = (end_date_inlaw - start_date_inlaw).days
age_of_account_johnson = (end_date_johnson - start_date_johnson).days
age_of_account_johnson_ira = (end_date_johnson_ira - start_date_johnson_ira).days

In [6]:
#Calculating monthly net deposit rates
monthly_inlaw_net_deposits = round(inlaw_net_deposits/age_of_account_inlaw, 2)
monthly_johnson_net_deposits = round(johnson_net_deposits/age_of_account_johnson, 2)
monthly_johnson_ira_net_deposits = round(johnson_ira_net_deposits/age_of_account_johnson_ira, 2)